In [1]:
import numpy as np
import tritonclient.http as httpclient
from tritonclient.utils import triton_to_np_dtype
from transformers import AutoTokenizer
import json

In [29]:
from transformers import AutoTokenizer

client = httpclient.InferenceServerClient(url="localhost:8000")
tokenizer = AutoTokenizer.from_pretrained("s-nlp/russian_toxicity_classifier")

In [30]:
import time

time.time()

1740774900.736378

In [34]:
import time

def preprocess_text(text):
    encoded_input = tokenizer(
        text,
        padding="max_length",
        truncation=True,
        max_length=128,
        return_tensors="np",
    )

    input_ids = encoded_input["input_ids"]
    attention_mask = encoded_input["attention_mask"]

    return {"input_ids": input_ids.astype(np.int64), "attention_mask": attention_mask.astype(np.int64)}

def process_inference_output(all_logits, threshold=0.5):
    probabilities = np.exp(all_logits[:, 1]) / np.sum(np.exp(all_logits), axis=1)
    predicted_class = probabilities > threshold
    return predicted_class

def inference_examples(texsts, model_name):
    infer_time = 0
    for text in texts:
        print(f"Text: {text}")
        inputs_np = preprocess_text(text)

        inputs = []
        for name, data in inputs_np.items():
            inputs.append(httpclient.InferInput(name, data.shape, datatype="INT64"))
            inputs[-1].set_data_from_numpy(data, binary_data=True)

        outputs = [httpclient.InferRequestedOutput("output", binary_data=True)]
        try:
            start_time = time.time()
            results = client.infer(model_name=model_name, inputs=inputs, outputs=outputs)
            infer_time += time.time() - start_time
            inference_output = results.as_numpy("output")
            predicted_class = process_inference_output(inference_output)
            print(f"Predicted Class: {predicted_class}")

        except Exception as e:
            print(f"Inference failed: {e}")
    print(f"Inference time: {infer_time}")

In [39]:
texts = [
    "Британцы любят чай. Я люблю чай. Я британец?",  # Non-toxic
    "А ну иди сюда ты говно собачье щаз я тебе выебу все что у тебя осталось.",  # Toxic
    "Британцы любят чай...",
]

In [40]:
%%time
inference_examples(texts, "onnx_fp32")

Text: Британцы любят чай. Я люблю чай. Я британец?
Predicted Class: [ True]
Text: А ну иди сюда ты говно собачье щаз я тебе выебу все что у тебя осталось.
Predicted Class: [ True]
Text: Британцы любят чай...
Predicted Class: [False]
Inference time: 0.45957469940185547
CPU times: total: 15.6 ms
Wall time: 464 ms


In [41]:
import pandas as pd
import numpy as np
from sklearn.metrics import roc_auc_score, precision_recall_curve, roc_curve, auc, precision_score
import matplotlib.pyplot as plt
from tqdm import tqdm

splits = {'train': 'train.jsonl', 'test': 'test.jsonl'}
df_test = pd.read_json("hf://datasets/AlexSham/Toxic_Russian_Comments/" + splits["test"], lines=True)

D:\E\Downloads\Anaconda\Lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [42]:
df_test.head()

,text,label
0,хорошо пошло!,0
1,"посмотрела, как будто дома побывала. как река ...",0
2,"отдам котят 1,5 месяца в добрые руки.",0
3,"0,5литровая баночка 200р стоит в таганроге. та...",0
4,речь шла о радужных зонтиках над верандой.,0


In [ ]:
from tqdm import tqdm

def inference_test(df, model_name):
    infer_time = 0
    predicted_labels = []
    for i in tqdm(range(len(df_test)), desc="Inferencing..."):
        text = df_test.iloc[i].text
        label = df_test.iloc[i].label
        inputs_np = preprocess_text(text)

        inputs = []
        for name, data in inputs_np.items():
            inputs.append(httpclient.InferInput(name, data.shape, datatype="INT64"))
            inputs[-1].set_data_from_numpy(data, binary_data=True)

        outputs = [httpclient.InferRequestedOutput("output", binary_data=True)]
        try:
            start_time = time.time()
            results = client.infer(model_name=model_name, inputs=inputs, outputs=outputs)
            infer_time += time.time() - start_time
            inference_output = results.as_numpy("output")
            predicted_class = process_inference_output(inference_output)
            predicted_labels.append(int(predicted_class[0]))
        except Exception as e:
            print(f"Inference failed: {e}")
    print(f"Inference time: {infer_time}")
    df["predicted"] = predicted_labels
    return df

In [ ]:
sample = df_test.sample(30)
inference_test(sample, "onnx_fp32")
sample

Traceback (most recent call last):
  File "C:\Users\ulyan\AppData\Roaming\Python\Python311\site-packages\gevent\libuv\loop.py", line 44, in python_queue_callback
    def python_queue_callback(self, watcher_ptr, revents):
    
KeyboardInterrupt


Inference failed: connection closed.
Inference failed: connection closed.
Inference failed: connection closed.
Inference failed: connection closed.
Inference failed: connection closed.
Inference failed: connection closed.
Inference failed: connection closed.
Inference failed: connection closed.
Inference failed: connection closed.
Inference failed: connection closed.
Inference failed: connection closed.
Inference failed: connection closed.
Inference failed: connection closed.
Inference failed: connection closed.
Inference failed: connection closed.
Inference failed: connection closed.
Inference failed: connection closed.
Inference failed: connection closed.
Inference failed: connection closed.
Inference failed: connection closed.
Inference failed: connection closed.
Inference failed: connection closed.
Inference failed: connection closed.
Inference failed: connection closed.
Inference failed: connection closed.
Inference failed: connection closed.
Inference failed: connection closed.
I

Inference failed: [Errno 10061] [WinError 10061] Подключение не установлено, т.к. конечный компьютер отверг запрос на подключение.
Inference failed: [Errno 10061] [WinError 10061] Подключение не установлено, т.к. конечный компьютер отверг запрос на подключение.
Inference failed: [Errno 10061] [WinError 10061] Подключение не установлено, т.к. конечный компьютер отверг запрос на подключение.
Inference failed: [Errno 10061] [WinError 10061] Подключение не установлено, т.к. конечный компьютер отверг запрос на подключение.
Inference failed: [Errno 10061] [WinError 10061] Подключение не установлено, т.к. конечный компьютер отверг запрос на подключение.
Inference failed: [Errno 10061] [WinError 10061] Подключение не установлено, т.к. конечный компьютер отверг запрос на подключение.
Inference failed: [Errno 10061] [WinError 10061] Подключение не установлено, т.к. конечный компьютер отверг запрос на подключение.
Inference failed: [Errno 10061] [WinError 10061] Подключение не установлено, т.к. к

In [ ]:
sample = df_test.sample(30)
inference_test(sample, "onnx_fp16")
sample